<a href="https://colab.research.google.com/github/rho-selynn/592-HW3/blob/Michael/592_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from torch import int64
import cv2 
import os 
import glob 
from google.colab.patches import cv2_imshow
import numpy as np
import torch
img_dir = "Images"
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
Images = [] 
for f1 in files: 
    img = cv2.imread(f1, 0) 
    Images.append(img)
Images = np.array(Images)
Images = (np.divide(Images, 255))
Images = Images.astype(float)

In [3]:
controlData = np.loadtxt('Robot Log/robot_log.csv', delimiter = ';', skiprows = 1, usecols = (1,2), dtype = float)
print(np.max(controlData[:,0]))
controlData[:,0] /= 15

15.0


In [24]:
from torch.utils.data import TensorDataset
ImagesTrain = torch.tensor(Images[0:799])
ImagesTrain = ImagesTrain.unsqueeze(1)
steerDataTrain = torch.tensor(controlData[0:799, 0])
trainSteerDataset = TensorDataset(ImagesTrain, steerDataTrain)
ImagesTest = torch.tensor(Images[800:999])
ImagesTest = ImagesTest.unsqueeze(1)
steerDataTest = torch.tensor(controlData[800:999, 0])
testSteerDataset = TensorDataset(ImagesTest, steerDataTest)
throttleDataTrain = torch.tensor(controlData[0:799, 1])
trainThrottleDataset = TensorDataset(ImagesTrain, throttleDataTrain)
throttleDataTest = torch.tensor(controlData[800:999, 1])
testThrottleDataset = TensorDataset(ImagesTest, throttleDataTest)

In [25]:
steerTrainDataLoader = torch.utils.data.DataLoader(trainSteerDataset, batch_size=128, shuffle=True)
steerTestDataLoader = torch.utils.data.DataLoader(testSteerDataset, batch_size=128, shuffle=False)
throttleTrainDataLoader = torch.utils.data.DataLoader(trainThrottleDataset, batch_size=128, shuffle=True)
throttleTestDataLoader = torch.utils.data.DataLoader(testThrottleDataset, batch_size=128, shuffle=False)

In [26]:
class LR(torch.nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.linear1 = torch.nn.Linear(160*320, 128) # W:784x128 , b:128x1 , parameters = [W,b]
        self.linear2 = torch.nn.Linear(128, 10)
        self.linear3 = torch.nn.Linear(10, 5)
        self.linear4 = torch.nn.Linear(5, 1)
    
    def forward(self, x):
        x = x.view(-1,160*320)
        transformed_x1 = self.linear1(x)
        transformed_x2 = self.linear2(transformed_x1)
        transformed_x3 = self.linear3(transformed_x2)
        transformed_x4 = self.linear4(transformed_x3)
        return transformed_x4

class LRthrottle(torch.nn.Module):
    def __init__(self):
        super(LRthrottle, self).__init__()
        self.conv2d1 = torch.nn.Conv2d(1, 1, 4, 1)
        self.linear1 = torch.nn.Linear(157, 56) # W:784x128 , b:128x1 , parameters = [W,b]
        self.linear2 = torch.nn.Linear(56, 10)
        self.linear3 = torch.nn.Linear(10, 5)
        self.linear4 = torch.nn.Linear(5, 1)
    
    def forward(self, x):
        transformed_x1 = self.conv2d1(x)
        transformed_x1 = transformed_x1.view(-1, 157)
        transformed_x2 = self.linear1(transformed_x1)
        transformed_x3 = self.linear2(transformed_x2)
        transformed_x4 = self.linear3(transformed_x3)
        transformed_x5 = self.linear4(transformed_x4)
        return transformed_x5

steerNet = LR()
steerOptimizer = torch.optim.Adam(steerNet.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
criterionThrottle = torch.nn.L1Loss()
throttleNet = LRthrottle()
throttleOptimizer = torch.optim.Adam(throttleNet.parameters(), lr=0.01)

In [27]:
#steering trainer
train_loss_history = []
test_loss_history = []
for epoch in range(30):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(steerTrainDataLoader):
        images, labels = data
        images = images.to(torch.float32)
        labels = labels.to(torch.float32)
        steerOptimizer.zero_grad()
        predicted_output = steerNet(images.float())
        predicted_output = predicted_output.to(torch.float32)
        loss = criterion(predicted_output, labels)
        loss.backward()
        steerOptimizer.step()
        train_loss += loss.item()
    for i, data in enumerate(steerTestDataLoader):
        with torch.no_grad():
            images, labels = data
            predicted_output = steerNet(images.float())
            loss = criterion(predicted_output, labels.long())
            test_loss += loss.item()
    train_loss = train_loss/len(steerTrainDataLoader)
    test_loss = test_loss/len(steerTestDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s finished with train loss %s and test loss %s'%(epoch, train_loss, test_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([71])) that is different to the input size (torch.Size([71, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, 

Epoch 0 finished with train loss 29888.365500054188 and test loss 13468.5849609375
Epoch 1 finished with train loss 6249.370152064732 and test loss 4651.25146484375
Epoch 2 finished with train loss 2887.6874340687477 and test loss 2682.5364990234375
Epoch 3 finished with train loss 954.2132045200893 and test loss 955.5394287109375
Epoch 4 finished with train loss 434.13907636914934 and test loss 98.57098007202148
Epoch 5 finished with train loss 198.30237034388952 and test loss 23.040464401245117
Epoch 6 finished with train loss 43.96292488915579 and test loss 84.43112564086914
Epoch 7 finished with train loss 34.37606067316873 and test loss 5.5967302322387695
Epoch 8 finished with train loss 19.42243378502982 and test loss 9.304011344909668
Epoch 9 finished with train loss 3.8289748515401567 and test loss 8.092686891555786
Epoch 10 finished with train loss 5.485945003373282 and test loss 0.09295963123440742
Epoch 11 finished with train loss 1.7052767404488154 and test loss 1.815754711

In [28]:
#throttle trainer
train_loss_history = []
test_loss_history = []
for epoch in range(40):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(throttleTrainDataLoader):
        images, labels = data
        images = images.to(torch.float32)
        labels = labels.to(torch.float32)
        throttleOptimizer.zero_grad()
        predicted_output = throttleNet(images.float())
        predicted_output = predicted_output.to(torch.float32)
        loss = criterion(predicted_output, labels)
        loss.backward()
        throttleOptimizer.step()
        train_loss += loss.item()
    for i, data in enumerate(throttleTestDataLoader):
        with torch.no_grad():
            images, labels = data
            predicted_output = throttleNet(images.float())
            loss = criterion(predicted_output, labels.long())
            test_loss += loss.item()
    train_loss = train_loss/len(throttleTrainDataLoader)
    test_loss = test_loss/len(throttleTestDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s finished with train loss %s and test loss %s'%(epoch, train_loss, test_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([40576, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([9827, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([71])) that is different to the input size (torch.Size([22507, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(

Epoch 0 finished with train loss 0.5520613959857396 and test loss 0.5269637778401375
Epoch 1 finished with train loss 0.42369059153965544 and test loss 0.40788932144641876
Epoch 2 finished with train loss 0.31031004658767153 and test loss 0.28120531886816025
Epoch 3 finished with train loss 0.25599920749664307 and test loss 0.2575986683368683
Epoch 4 finished with train loss 0.2396364084311894 and test loss 0.25596698373556137
Epoch 5 finished with train loss 0.23670912427561625 and test loss 0.2475515678524971
Epoch 6 finished with train loss 0.23785267983164107 and test loss 0.24707268178462982
Epoch 7 finished with train loss 0.23689691722393036 and test loss 0.24941980838775635
Epoch 8 finished with train loss 0.2350306170327323 and test loss 0.2487040013074875
Epoch 9 finished with train loss 0.2327340351683753 and test loss 0.24898315221071243
Epoch 10 finished with train loss 0.23241570804800307 and test loss 0.25048353523015976
Epoch 11 finished with train loss 0.23643982836178

In [46]:
from torch import int64
import cv2 
import os 
import glob 
from google.colab.patches import cv2_imshow
import numpy as np
import torch
img_dir = "Images"
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
Images = [] 
for f1 in files: 
    img = cv2.imread(f1, 0) 
    Images.append(img)
Images = np.array(Images)
Images = (np.divide(Images, 255))
Images = Images.astype(float)
Images = torch.tensor(Images)
Images = Images.unsqueeze(1)
Images = Images.unsqueeze(1)

for i in np.linspace(1, 300, 100).astype(int):
  predicted_output_steer = steerNet(Images[i].float())
  predicted_output_throttle = throttleNet(Images.float()[i])
  correct_steer = controlData[i,0]
  correct_throttle = controlData[i,1]
  print ("correct steer: " + str(correct_steer) + " predicted steer: " + str(predicted_output_steer))
  print ("correct throttle: " + str(correct_throttle) + " predicted throttle: " + str(predicted_output_throttle[i]))

correct steer: 0.0 predicted steer: tensor([[0.1181]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([0.4005], grad_fn=<SelectBackward0>)
correct steer: 0.0 predicted steer: tensor([[-0.0376]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([0.4010], grad_fn=<SelectBackward0>)
correct steer: 0.0 predicted steer: tensor([[-0.0918]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([0.4006], grad_fn=<SelectBackward0>)
correct steer: 0.0 predicted steer: tensor([[-0.0258]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([0.4008], grad_fn=<SelectBackward0>)
correct steer: 0.0 predicted steer: tensor([[0.0113]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([0.4017], grad_fn=<SelectBackward0>)
correct steer: 0.0 predicted steer: tensor([[-0.0591]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([0.4001], grad_fn=<SelectBackwa